## 1. Library Importing

In [36]:
import nltk # 텍스트 데이터를 처리
import numpy as np # 말뭉치를 배열로 표현
import random
import operator
import string # 표준 파이썬 문자열을 처리
import re
from sklearn.metrics.pairwise import cosine_similarity # 이를 나중에 사용하여 두 개의 문장이 얼마나 비슷한지를 결정합니다.
from sklearn.feature_extraction.text import TfidfVectorizer # Experience 2에서 단어 가방을 만드는 함수를 만들었던 것을 기억하십니까? 이 함수는 같은 일을 합니다!
import re

from google.colab import drive

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # LogisticRegression import 추가

import os
import numpy as np
from google.colab import drive

## 2. Data Uploading

In [37]:
  # Colab에서 Google 드라이브를 마운트하기 위한 라이브러리

# 구글 드라이브 마운트
drive.mount('/content/drive')

def loadfile(path):
    X = []
    Y = []

    for label in ('0', '25', '50', '75', '100'):
        print("Loading text files for the label: " + label)

        label_path = os.path.join(path, label)
        for filename in os.listdir(label_path):
            if filename.endswith('.txt'):
                with open(os.path.join(label_path, filename), 'r', encoding='utf-8') as file:
                    text = file.read()

                # 레이블을 숫자로 변환
                if label == '0':
                    Y.append(0)
                elif label == '25':
                    Y.append(1)
                elif label == '50':
                    Y.append(2)
                elif label == '75':
                    Y.append(3)
                elif label == '100':
                    Y.append(4)

                # 텍스트 데이터를 X에 추가
                X.append(text)

    X = np.array(X)
    Y = np.array(Y)

    return X, Y

# 경로를 구글 드라이브 경로로 변경
directory_path = '/content/drive/MyDrive/likeability_Son/'

# loadfile 함수 호출
X, Y = loadfile(directory_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading text files for the label: 0
Loading text files for the label: 25
Loading text files for the label: 50
Loading text files for the label: 75
Loading text files for the label: 100


In [38]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (500,)
Y shape: (500,)


In [39]:
def count_txt_files(path):
    file_counts = {}

    for label in ('0', '25', '50', '75', '100'):
        label_path = os.path.join(path, label)
        txt_files = [filename for filename in os.listdir(label_path) if filename.endswith('.txt')]
        file_counts[label] = len(txt_files)

    return file_counts

# 경로를 구글 드라이브 경로로 변경
directory_path = '/content/drive/MyDrive/likeability_Son/'

# .txt 파일 수 확인
file_counts = count_txt_files(directory_path)

# 결과 출력
for label, count in file_counts.items():
    print(f"Number of .txt files in label {label}: {count}")

Number of .txt files in label 0: 100
Number of .txt files in label 25: 100
Number of .txt files in label 50: 100
Number of .txt files in label 75: 100
Number of .txt files in label 100: 100


## 3. Data Refining

In [40]:
def clean_str(string):
    string = re.sub(r"[^가-힝A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r":", " : ", string)  # 콜론은 다른 문자로 대체하지 않음
    string = re.sub(r"\s{2,}", " ", string)

    return string.lower()

In [41]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

def clean_text(text):
    text = text.lower()  # 소문자화
    text = re.sub(r'[.:]', '', text)  # 특정 기호 제거
    text = re.sub(r'\s+', ' ', text)  # 공백 정규화
    return text

In [42]:
from datetime import datetime
import re
import numpy as np
import os

def load_data_and_labels(directory_path):
    conversations = []
    labels = []

    for label in ('0', '25', '50', '75', '100'):
        label_path = os.path.join(directory_path, label)
        for filename in os.listdir(label_path):
            if filename.endswith('.txt'):
                with open(os.path.join(label_path, filename), 'r', encoding='utf-8') as file:
                    file_content = file.read()
                    conversations.append(file_content)
                    labels.append(int(label))

    return conversations, labels

directory_path = '/content/drive/MyDrive/likeability_Son/'
conversations, labels = load_data_and_labels(directory_path)

print(f'Number of conversations: {len(conversations)}')
print(f'Number of labels: {len(labels)}')


Number of conversations: 500
Number of labels: 500


In [43]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (500,)
Y shape: (500,)


## 4. 호감도 가중치 증가 함수

4-1. 이모지 사용 함수

In [44]:
def count_emojis(text):
    emoji_pattern = re.compile('['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        ']', flags=re.UNICODE)
    return len(emoji_pattern.findall(text))


4.2. 대화 양방향성 관련 함수

In [45]:
def check_bidirectional_conversation(text):
    a_contributions = len(re.findall(r'A:', text))
    b_contributions = len(re.findall(r'B:', text))
    return a_contributions > 0 and b_contributions > 0

4-3. 답장 속도 기반 호감도 예측

In [46]:
from datetime import datetime
import re
import numpy as np

# 대화 데이터를 .txt 파일에서 불러오기
conversation = []
for label in ('0', '25', '50', '75', '100'):
    label_path = os.path.join(directory_path, label)
    for filename in os.listdir(label_path):
        if filename.endswith('.txt'):
            with open(os.path.join(label_path, filename), 'r', encoding='utf-8') as file:
                conversation.extend(file.readlines())

# 호감도 레이블 생성 함수
def create_likeability_labels(conversation):
    response_times = []
    a_message_time = None  # A의 메시지 시간
    b_message_time = None  # B의 메시지 시간

    for line in conversation:
        if 'A:' in line:
            time_str = re.search(r'\((\d{2}):(\d{2})\)', line)
            if time_str:
                hours, minutes = map(int, time_str.groups())
                a_message_time = hours * 60 + minutes

        if 'B:' in line and a_message_time is not None:
            time_str = re.search(r'\((\d{2}):(\d{2})\)', line)
            if time_str:
                hours, minutes = map(int, time_str.groups())
                b_message_time = hours * 60 + minutes
                response_time = b_message_time - a_message_time
                response_times.append(response_time)

    # 호감도 레이블 생성
    likeability_labels = []
    for response_time in response_times:
        if response_time <= 10:
            likeability = 100
        elif response_time <= 30:
            likeability = 75
        elif response_time <= 60:
            likeability = 50
        elif response_time <= 180:
            likeability = 25
        else:
            likeability = 0
        likeability_labels.append(likeability)

    return likeability_labels

# 대화 데이터를 기반으로 호감도 레이블 생성
likeability_labels = create_likeability_labels(conversation)

In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import re

# 대화 데이터 불러오기 및 정제
conversations = []
for label in ('0', '25', '50', '75', '100'):
    label_path = os.path.join(directory_path, label)
    for filename in os.listdir(label_path):
        if filename.endswith('.txt'):
            with open(os.path.join(label_path, filename), 'r', encoding='utf-8') as file:
                for line in file:
                    conversations.append(clean_str(line))

# 추가 특성 추출 (이모티콘 사용 및 양방향 대화)
emoji_counts = [count_emojis(conv) for conv in conversations]
bidirectional = [check_bidirectional_conversation(conv) for conv in conversations]

# 토크나이징 및 시퀀스 변환
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(conversations)
sequences = tokenizer.texts_to_sequences(conversations)

# 패딩
data = pad_sequences(sequences, maxlen=100)

# 호감도 레이블 생성
likeability_labels = create_likeability_labels(conversations)

# 데이터 분할 (Train:Test:Validation = 7:2:1)
X_train, X_temp, Y_train, Y_temp = train_test_split(data, likeability_labels, test_size=0.3, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=1/3, random_state=42)


ValueError: ignored

## 4.

In [11]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.regularizers import l1_l2

model = Sequential()
model.add(Embedding(10000, 64, input_length=100))
model.add(LSTM(1024, return_sequences=True, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(LSTM(512, return_sequences=True, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(LSTM(256, return_sequences=True, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
model.add(Dropout(0.5))
model.add(LSTM(64, kernel_regularizer=l1_l2(l1=0.01, l2=0.01))) # 마지막 LSTM 레이어
model.add(Dense(5, activation='softmax')) # 5개의 클래스 (0, 25, 50, 75, 100)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
from sklearn.model_selection import train_test_split

# 데이터 분할
# 먼저 전체 데이터를 Train (70%)과 나머지 (30%)로 분할
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)

# 나머지 데이터를 다시 Test (2/3 of 30%)와 Validation (1/3 of 30%)으로 분할
# 최종 비율: Train 70%, Test 20%, Validation 10%
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=1/3, random_state=42)

# 모델 훈련
history = model.fit(
    X_train, Y_train,
    epochs=20,
    batch_size=4,
    validation_data=(X_val, Y_val)
)

# 훈련 결과 출력
print("Training Loss: ", history.history['loss'])
print("Training Accuracy: ", history.history['accuracy'])
print("Validation Loss: ", history.history['val_loss'])
print("Validation Accuracy: ", history.history['val_accuracy'])


Epoch 1/20


ValueError: ignored

## 5.평가지표

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return accuracy, f1, precision, recall

# 테스트 데이터에 대한 예측 수행
# 여기서 'model'은 이미 훈련된 모델을 나타냅니다.
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# 모델 평가
accuracy = accuracy_score(Y_test, y_pred_classes)
f1 = f1_score(Y_test, y_pred_classes, average='weighted')
precision = precision_score(Y_test, y_pred_classes, average='weighted')
recall = recall_score(Y_test, y_pred_classes, average='weighted')

# 결과 출력
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

4/4 [==============================] - 1s 4ms/step
Accuracy: 0.28
F1 Score: 0.12250000000000001
Precision: 0.07840000000000001
Recall: 0.28


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 6. 시각화 그래프

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(metrics):
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(metrics['accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')

    plt.subplot(1, 2, 2)
    plt.plot(metrics['loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()

In [ ]:
# plot_metrics(history.history)

In [ ]:

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# # LSTM 모델 구축
# model = Sequential()
# model.add(Embedding(10000, 64, input_length=100))
# model.add(LSTM(128))
# model.add(Dense(5, activation='softmax'))
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# from tensorflow.keras.layers import GRU

# # GRU 모델 구축
# model = Sequential()
# model.add(Embedding(10000, 64, input_length=100))
# model.add(GRU(128))
# model.add(Dense(5, activation='softmax'))
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# BERT 모델 구현 예시 코드


In [ ]:

# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# import numpy as np

# # 가정: `texts`는 대화 데이터셋의 원본 텍스트를 포함하는 리스트
# # 토크나이징 및 시퀀스로 변환
# tokenizer = Tokenizer(num_words=10000)
# tokenizer.fit_on_texts(texts)
# sequences = tokenizer.texts_to_sequences(texts)

# # 패딩
# data = pad_sequences(sequences, maxlen=100)

# # 데이터셋을 훈련, 테스트, 검증 세트로 분할
# X_train, X_temp, Y_train, Y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
# X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=2/3, random_state=42)
